In [1]:
import pandas as pd
from collections import defaultdict
import spacy

In [34]:
# Load dataset
file_path = 'assets/dataset/Resume.csv'
data = pd.read_csv(file_path)

In [35]:
# Load spaCy model
nlp = spacy.load(".venv/Lib/site-packages/en_core_web_sm/en_core_web_sm-3.8.0")

In [ ]:
# Fungsi untuk mengekstrak keterampilan dari teks
def extract_skills(text):
    doc = nlp(text)
    skills = []
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"] and not token.is_stop:  # Hanya mengambil kata benda dan proper noun
            skills.append(token.text.lower())
    return skills

In [37]:
# Dictionary untuk mengelompokkan keterampilan berdasarkan kategori pekerjaan
job_skills = defaultdict(list)

In [38]:
# Proses setiap baris dalam dataset
for index, row in data.iterrows():
    category = row['Category']
    resume_text = row['Resume_str']
    
    # Ekstrak keterampilan dari resume
    skills = extract_skills(resume_text)
    
    # Tambahkan keterampilan ke kategori pekerjaan
    job_skills[category].append(set(skills))

In [39]:
# Membuat tabel baru untuk `Job Category` dan keterampilan
job_table = []

In [43]:
for category, skills_list in job_skills.items():
    # Temukan keterampilan minimum (muncul di semua resume dalam kategori ini)
    mandatory_skills = set.intersection(*skills_list) if len(skills_list) > 0 else set()
    
    # Temukan keterampilan tambahan (muncul di beberapa resume tetapi tidak semua)
    all_skills = set.union(*skills_list)
    additional_skills = all_skills - mandatory_skills
    
    # Tambahkan data ke tabel
    job_table.append({
        "Job Category": category,
        "Mandatory Skills": list(mandatory_skills),
        "Additional Skills": list(additional_skills)
    })

In [44]:
# Konversi tabel menjadi DataFrame
job_df = pd.DataFrame(job_table)

In [45]:
# Simpan hasil ke file baru
output_path = 'Job_Category_Requirements.csv'  # Simpan hanya sekali
job_df.to_csv(output_path, index=False)
print(f"Tabel job requirements berhasil disimpan ke {output_path}")

Tabel job requirements berhasil disimpan ke Job_Category_Requirements.csv
